In [1]:
# ✅ Step 1: Set cache paths to use external drive
import os
external_drive = "/Volumes/My Passport for Mac"
os.environ["HF_HOME"] = f"{external_drive}/.hf_cache"
os.environ["MFLUX_CACHE_DIR"] = f"{external_drive}/.mflux_cache"

# ✅ Step 2: Log in to Hugging Face
from huggingface_hub import login
login(token="________________________")  # <-- replace with your token

# ✅ Step 3: Load the model with quantization
from mflux.flux.flux import Flux1
flux = Flux1.from_name(model_name="dev", quantize=8)  # 8-bit quantization

# ✅ Step 4: Save the model to external drive
save_path = f"{external_drive}/flux_weights/"
os.makedirs(save_path, exist_ok=True)
flux.save_model(save_path)


HTTPError: Invalid user token.

In [ ]:

from mflux.config.model_config import ModelConfig
from mflux.flux.flux import Flux1

# Step 1: Create proper ModelConfig
model_config = ModelConfig(
    alias="schnell_local",
    model_name="schnell",
    base_model="schnell_base",  # adjust if needed
    controlnet_model=None,
    custom_transformer_model=None,
    num_train_steps=1000,
    max_sequence_length=128,
    supports_guidance=True,
    requires_sigma_shift=False,
    priority=1
)



# Step 2: Initialize Flux1 and point to local path
flux = Flux1(model_config=model_config, local_path="/Volumes/My Passport for Mac/flux_weights/")


# Optional: set dtype or other model config params if needed
# flux.set_dtype("float32")  # Just an example if you need control

# Load sharper, faster model
#flux = Flux1.from_name("schnell", quantize=8)
from mflux.config.config import Config
config = Config()


# Improved prompt (optional enhancements)
prompt = (
"A magical night-time alpine landscape under a starry sky, with a glowing crescent moon casting silvery light over snow-covered mountains and a tranquil pine forest. A small wooden cabin with warm lights glows gently in the distance, reflected in a crystal-clear mountain lake. The stars twinkle above in a deep indigo sky with the Milky Way faintly visible. Ultra-detailed, cinematic atmosphere, soft fog, 8K photo-realism, fantasy mood with natural realism."
)
# Seed for reproducibility
seed = 42

# Actually generate the image
image = flux.generate_image(seed, prompt, config)


from PIL import Image
from IPython.display import display

# Resize to 512x512
resized_img = image.image.resize((512, 512))

# Display the resized image
display(resized_img)



  0%|          | 0/4 [00:00<?, ?it/s]

In [1]:
import gradio as gr
from mflux.config.model_config import ModelConfig
from mflux.config.config import Config
from mflux.flux.flux import Flux1
from PIL import Image


# Step 1: Prepare model
model_config = ModelConfig(
    alias="schnell_local",
    model_name="schnell",
    base_model="schnell_base",
    controlnet_model=None,
    custom_transformer_model=None,
    num_train_steps=1000,
    max_sequence_length=128,
    supports_guidance=True,
    requires_sigma_shift=False,
    priority=1
)

config = Config()
flux = Flux1(model_config=model_config, local_path="/Volumes/My Passport for Mac/flux_weights/")

# Step 2: Define generation function
def generate_and_resize(prompt, seed, size):
    image_obj = flux.generate_image(seed, prompt, config)
    resized = image_obj.image.resize((size, size))
    return resized

# Step 3: Launch Gradio app
demo = gr.Interface(
    fn=generate_and_resize,
    inputs=[
        gr.Textbox(label="Prompt", value="A magical night-time alpine landscape..."),
        gr.Number(label="Seed", value=42),
        gr.Slider(256, 1048, value=512, label="Resize (px)")
    ],
    outputs=gr.Image(type="pil"),
    title="Flux1 Fantasy Image Generator",
    description="Generated imagination ✨"
)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1a3e28ff203a616875.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
